In [1]:
import os
import logging
import numpy as np
from ase.calculators.vasp import Vasp2
from ase.io import read

out_dir = "test"
if not os.path.isdir(out_dir+"log"):
    os.system("mkdir -p {}/log".format(out_dir))

logging.basicConfig(format='%(asctime)s : %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p',
               filename=out_dir+"/log/valet.log", filemode="w",level=logging.INFO)  

In [2]:
class ValetJob:
    def __init__(self,POSCAR_init):
        
        if os.path.isfile(POSCAR_init):
            self.poscar_init = POSCAR_init
        else:
            logging.error("Cannot locate POSCAR file as {}".format(POSCAR_init))
            print("ERROR: See log for error.")
            exit()
        self.encut = None
        self.poscar_vol = None
        self.poscar_opt = None
        self.band_structure = None
        self.band_gap = None
        
    def set_encut_autotune_params(self,tol=None,max_steps=None,start_encut=None,encut_step=None):
        stop = False
        ens = []
        cnt = 0
        return tol,stop,cnt,max_steps,start_encut,encut_step,ens
    
    def check_stop(self,x,tol):
        perc_diff = lambda x,y: np.abs(np.abs(x-y)/((x+y)/2))*100
        if len(x) < 2:
            return False
        elif perc_diff(x[-1],x[-2]) < tol:
            return True
        else: 
            return False
        
    def do_encut_autotune(self,tol=1E-4,max_steps=32,start_encut=200,encut_step=25,retune=False):
        logging.info("Commencing with ENCUT autotune.")
        if (self.encut == None) or (retune):
            sys = read(self.poscar_init)
            calc = Vasp2(xc='PBE',kpts=(4,4,4),directory=out_dir,atoms=sys)
            tol,stop,cnt,max_steps,ENCUT,ENCUT_step,ens = self.set_encut_autotune_params(tol=tol,max_steps=max_steps,
                                                                                         start_encut=start_encut,
                                                                                         encut_step=encut_step)

            print("Autotuning ENCUT...")
            while not stop:
                if 'error' in locals():
                    del error
                if cnt < max_steps:
                    logging.debug("Count {}".format(cnt))
                    calc.set(encut=ENCUT)
                    ens.append(sys.get_potential_energy())
                    #print(ENCUT)
                    stop = self.check_stop(ens,tol)
                    ENCUT += ENCUT_step
                    cnt += 1
                else:
                    logging.debug("Maximum autotune steps reached.")
                    error = "Maximum autotune steps reached."
                    tip = "Tip: Increase either start_encut, encut_step, or max_steps. Decrease tol."
                    stop = True

            os.system("rm {}/*".format(out_dir))
            try:
                print(error)
                print(tip)
                self.encut = None
            except NameError:
                print("\tENCUT auto-tuned to: {}".format(ENCUT))
                logging.info("Completed ENCUT autotune.")
                self.encut = ENCUT
            print("Done.")  
        else:
            print("Already tuned.")
        logging.info("Completed ENCUT autotune.")

In [3]:
a = ValetJob("POSCAR.mp-27_Si")

In [4]:
print(a.encut)
a.do_encut_autotune()

None
Autotuning ENCUT...
	ENCUT auto-tuned to: 725
Done.


In [5]:
a.do_encut_autotune()

Already tuned.


In [8]:
a.do_encut_autotune(start_encut=700,encut_step=10,retune=True)

Autotuning ENCUT...
	ENCUT auto-tuned to: 720
Done.
